In [15]:
import asyncio
import aiohttp
import pandas as pd
from datetime import datetime, timedelta
from my_funnel_utils import load_api_tokens

In [ ]:
async def get_funnel_v3(days_num : int, account : str, api_token : str):
    """ Функция для получения статистических данных по воронке продаж ВБ"""
    products_list = []
    headers = {'Authorization': api_token}
    # Задержка при 429 ошибке
    delay = 20
    start = end = datetime.now()-timedelta(days=days_num)
    url = "https://seller-analytics-api.wildberries.ru/api/analytics/v3/sales-funnel/products"

    # Количество попыток получить данные по запросу
    max_attempts = 10
    # Начальное кол-во попыток
    attempt = 0
    # Количество карточек в ответе
    limit = 50
    # Сколько элементов пропустить. Например, для значения 10 ответ начнется с 11 элемента
    offset = 0
    # Создание сессии для асинхронного http запроса
    async with aiohttp.ClientSession(headers=headers) as session:
        # Цикл из 10 попыток для получения данных
        while attempt != max_attempts:
            # Тело POST-запроса
            payload = {
                        "selectedPeriod": {
                        "start": start.strftime('%Y-%m-%d'),
                        "end": end.strftime('%Y-%m-%d')
                        },
                        "limit": limit
                            }
            try:
                # Запуск сессии
                async with session.post(url, json=payload) as res:
                    if res.status == 200:
                        # Сбрасываем счетчик при успешном запросе
                        attempt = 0
                        # Асинхронный http запрос
                        data = await res.json()
                        products = data['data']['products']
                        for product in products:
                            product["account"] = account
                        products_list.extend(products)
                        print(f"✅ Получены данные по {len(products_list)} товарам по ЛК: {account}")
                        await asyncio.sleep(delay)
                    # Обработка неправильного запроса
                    elif res.status == 400:
                        # Создаем запрос
                        error_data = await res.json()
                        # Сохраняем в переменную. Пытаемся получить данные по ключу message. Если такого ключа нет, выведем 'Неправильный запрос'
                        error_detail = error_data.get('message', 'Неправильный запрос')
                        print(f"⚠️ Ошибка 400 для аккаунта {account}: {error_detail}")
                        return None 
                    # Обработка ошибки авторизации данных 
                    elif res.status == 401:
                        print(f"⚠️ Ошибка 401 для аккаунта {account}: Не авторизован")
                        return None
                    # Обработка запрета на получение данных
                    elif res.status == 403:
                        # Создаем запрос
                        error_data = await res.json()
                        # Сохраняем в переменную. Пытаемся получить данные по ключу message. Если такого ключа нет, выведем 'Неправильный запрос'
                        error_detail = error_data.get('message', 'Доступ запрещен')
                        print(f"⚠️ Ошибка 403 для аккаунта {account}: {error_detail}")
                        return None
                    # Обработка лимита запросов
                    elif res.status == 429:
                        error_data = await res.json()
                        error_detail = error_data.get('detail', 'Слишком много запросов')
                        print(f"⚠️ Ошибка 429 для аккаунта {account}: {error_detail}")
                        print(f"Лимит: 3 запроса в 1 минуту на аккаунт. ждем {delay}")
                        # Ждем перед повторной попыткой
                        await asyncio.sleep(delay) 
                        attempt += 1
                        continue
                    else:
                        print('Нет данных по товарам')
                        attempt += 1
            except aiohttp.ClientError as err:
                print(f'Сетевая ошибка {err}')
            except Exception as e:
                print(f'Неожиданная ошибка {e}')
            if attempt == max_attempts:
                break
    if products_list:
        df = pd.DataFrame(products_list)
        print(f"🟢 Завершено получение статистики по воронки продаж для {account}")
        return df
    else:
        print('Не удалось получить статистику по воронки продаж')
        return None
    
async def main():
    # Создаем задачник для получения данных о поставках по всем аккаунтам асинхронно
    tasks = [get_funnel_v3(1, account, api_token) for account, api_token in load_api_tokens().items()]
    res = await asyncio.gather(*tasks)
    return pd.concat(res)

In [ ]:
result = await main()